## Run this line before running the game the first time

In [ ]:
pip install PyGithub

In [ ]:
"""
Docstring: The player must look for the exit of labyrinthic forest, he meets 
random characters from the online game database. 

If the player wins, he can add a new character, which is
added to the database online and once the author of the game check it for
typos and correctness it is added to the game for all the following players.

Known bugs:
- The player can submit characters that don’t work. the author needs to check
all the characters (e.g. the user input “Tall and brunette” as description of 
the appearance of the character but the game should say “She’s tall and brunette”.)

- You can encounter the same character twice on the same game and he will ask 
you the same question.

- It may take some time to update the new character after the online database 
is updated, probably due to the cache stored offline.

"""


import numpy as np
import random
import matplotlib.pyplot as plt
import urllib
from github import Github
from datetime import datetime
import time

#############################################################################

g = Github("advgame2020@gmail.com","37433f05fbbca2617280ea16b386b1dfa95dd6ff")

#############################################################################

# Seed for software development
# np.random.seed(42)

#############################################################################

# Class used to store the data concerning 1 character of the game
class Character:
    # class variables
    info = [["name",""],
            ["apperance",""],
            ["question",""],
            ["option_1",""],
            ["option_2",""],
            ["option_3",""],
            ["option_4",""],
            ["correct_answer",""],
            ["win_msg",""],
            ["lose_msg",""],
            ["author",""]]
    
    # class constructor, resetting the variables    
    def __init__(self):
        self.info = [["name",""],
                    ["apperance",""],
                    ["question",""],
                    ["option_1",""],
                    ["option_2",""],
                    ["option_3",""],
                    ["option_4",""],
                    ["correct_answer",""],
                    ["win_msg",""],
                    ["lose_msg",""],
                    ["author",""]]
        return




#############################################################################

# Class used to store all the characters from the present labyrinth
class Characters_pool:
    
    # class variables
    characters_pool = []
    n_character = -1
    
    # class constructor, resetting the variables
    def __init__(self):
        self.characters_pool = []
        self.n_character = -1
        return
    
    # Method to download characters from the cloud (Github)
    def Load(self):
        n_character = -1 #Keeps track of the number of characters
        n_info = 0
        self.characters_pool = []
        
        # Retrieving the data from https://github.com/Nicolab367/text_adventure_game/blob/main/characters.txt
        url = "https://raw.githubusercontent.com/Nicolab367/text_adventure_game/main/characters.txt"
        file = urllib.request.urlopen(url)
        
        # Decoding the data and adding them to a organised list
        for line in file:
            text = line.decode("utf8")
            text = text.split("#")
            
            # The '@' indicates the start of data concerning a new character
            # In this way we can change the number of characteristics of a character
            # without changing the Load() method
            if(text[0]=="@\n"): 
                
                # Indicator the keep track of the n-th info we are extracting
                n_info = 0 
                
                # Creating a new character empty
                new = Character() 
                
                # Appending the new empty character to the characters pool
                self.characters_pool.append(new) 
                
                #Keeping track of the number of characters
                n_character += 1
                
            # If it's not a new character we continue to add data to the current
            # character
            else:
                #Cancelling white spaces before and after the string
                text[1] = text[1].strip()
                
                self.characters_pool[n_character].info[n_info][1] = text[1]
                n_info += 1

    

    # Method that puts all the characters data in a string, separated with 
    # the special sperators '@' and '#'
    # The text is also formated in a way that is easily read and edited with
    # a text editor.
    def Format_database(self):
        database = ""
        for character in self.characters_pool:
            database = database + "@\n"
            for lines in range(len(character.info)):
                database = database + character.info[lines][0]
                
                if(len(character.info[lines][0])<8):
                    database = database + "\t\t\t" + "#"
                else:
                    database = database + "\t\t" + "#"
                    
                database = database + character.info[lines][1] + "#\n"
        return database
        
    # Saving the characters on another account, not my first one, for safety
    # purposes, the final version of the game should be compiled and use
    # a server request to a private server with player authentication
    def Save(self):
        # Get the account user owner of the reposity
        repo = g.get_user().get_repo("advgame")
        
        # Get content of the current database of characters
        contents = repo.get_contents("characters.txt", ref="main")
        time = datetime.now()
        ### Make backup of previous database with a unique and identifiable name
        fileName = "backup/backup #" + str(time) + "   " +  str(np.random.randint(1,100000))
        contents = repo.get_contents("characters.txt", ref="main")
        repo.create_file(fileName, "Nicola" ,contents.decoded_content)
        
        ### Update the primary database (i.e. the file "characters.txt") ###            
        repo.update_file(contents.path, "NICOLA", self.Format_database(), contents.sha,branch="main")
        

#############################################################################

###### class Labyrinth:
class Labyrinth:
    
    # Minimum and maximum waiting time when wandering, max_wait must always be 
    # greater then min_wait 
    min_wait = 2
    max_wait = 6
    
    # Player name 
    player_name = ""
    
    # Chance of winning, everytime the player respond to a question correctly 
    # it increases
    win_chance = 0
    
    # Percentage of how much the chance of winning increases everytime the player
    # answer correctly
    win_chance_increase = 0.334
    
    # Chance of losing, everytime the player respond to a question uncorrectly
    # it increases
    lose_chance = 0
    
    # Percentage of how much the chance of losing increases everytime the player
    # answe uncorrectly
    lose_chance_increase = 0.334
    
    # Boolean status of the player, when a player faints the game is ended.
    # The game always ends with a faint, it does not necessarily mean that
    # the player has failed
    fainted = False
    
    # Character that player encounters
    current_char = Character()
    
    # Pool of characters of the labyrinth, it is loaded from the online database
    pool = Characters_pool()
    
    
    # Constructor method that creates an instance of the class when called
    def __init__(self):
        # Download pool of characters from Github
        self.pool = Characters_pool()
        self.pool.Load()
        return

    #Main function of the game, when it ends the game is ended
    def Start(self):
        
        # Method to create spacing between output lines
        self.Spacing(1)
        
        print("""
You wake up quite confused, you are laying on the grass, as
soon as you open your eyes you hear a firm voice from above you:""")
        
        self.Spacing(1)
        
        correct_name = False
        # Asking for user name or nickname (we want the user to insert numbers if he wants)
        # Everything is correct expect for leaving it blank
        while (correct_name != True):
            print(""" <Uknown> 'What's your name?' """)
            self.player_name = input(prompt="> ")
            if self.player_name != "":
                correct_name = True
            else:
                self.Spacing(0)
                print(" <Uknown> 'I need your name or you cannot enter'")

        self.Spacing(1)
        
        #Introduction of the game
        print("""
You are outside, it's pitch dark and there's a thick fog. 
You can see some trees around you and many leaves and branches covering the sky.
It seems a really thick forest.

You see a small guy with a long black coat and a hoodie in front of you
filling out a form with your name, you imagine.

After writing he starts reading:
"Welcome Player, we will be watching you, look for someone in the forest
and answer correctly to their questions in order to escape."

As soon as he finishes to read he runs away and after a few steps he
dissolve into the fog.
        """)
        
        print("You decide to start looking for a way out of the forest")
        
        # The player continues to wander, looking for the exit until it faints
        # Even if the player finds the exit the master of the labyrinth will make
        # him faint. Nobody knows what happen next
        self.Continue(1)
        while (not self.fainted):
            self.Wander()
        


    # Encounter with a random character
    def Rand_Encounter(self):
        
        #Picking a random character from the pool
        r_character = np.random.choice(self.pool.characters_pool)
        
        # Picking a random number from 0 to 1 to calculate randomly if the player
        # has won or not
        r_win_lose = np.random.random()
        
        # Checking if the player has won, based on the random variable r_win_lose
        if (self.win_chance>r_win_lose):
            self.Win()
            return

        # Checking if the player has won, based on the random variable r_win_lose
        if (self.lose_chance>r_win_lose):
            self.Fail()
            return
        
        self.Spacing(1)
        
        # Print character's apperance
        print(r_character.info[1][1])
        
        self.Continue(0)
        
        
        # Formating the possible answers proposed to the player in a way that
        # are easily if the answer matches one of the proposed answers
        opt_1 = "1) " + str(r_character.info[3][1]).lower()
        opt_2 = "2) " + str(r_character.info[4][1]).lower()
        opt_3 = "3) " + str(r_character.info[5][1]).lower()
        opt_4 = "4) " + str(r_character.info[6][1]).lower()
        
        # Variable to keep track of the chosen answer
        n_answer = 0
        
        # While the answer it's not one of the accepted answers it doesn't continue
        while(n_answer == 0):
            
            self.Spacing(1)
            
            # Print character's question
            print(f"""
<{r_character.info[0][1]}> '{r_character.info[2][1]}' \n\n""")
            
            # Print answer's options
            print(f"""
        {opt_1.title()}
        {opt_2.title()}
        {opt_3.title()}
        {opt_4.title()}
                   """)
            
            # Ask for an answer to the player
            answer = str(input(prompt = "What's your answer?")).lower()

            # Check if the answer given matches one of the proposed answers
            # and it's different from all the others possible answers
            if (answer in opt_1) and (answer not in opt_2) and (answer not in opt_3) and (answer not in opt_4):
                n_answer = 1
            elif (answer in opt_2) and (answer not in opt_1) and (answer not in opt_3) and (answer not in opt_4):
                n_answer = 2
            elif (answer in opt_3) and (answer not in opt_2) and (answer not in opt_1) and (answer not in opt_4):
                n_answer = 3
            elif (answer in opt_4) and (answer not in opt_2) and (answer not in opt_3) and (answer not in opt_1):
                n_answer = 4
            else:
                self.Spacing(1)
                print("""
That is not any of the possible answers, please enter a correct answer
                      """)
        
        self.Spacing(1)
                
        # Check if the answer given is correct
        if str(n_answer) == r_character.info[7][1]:
            
            # If the answer is correct the character will guide the player 
            # toward the exit for a while, increasing his chances to find the
            # exit
            print(r_character.info[8][1])
            print("Follow me! I'll guide you for a while",end="")
            self.Continue(0)
            self.Spacing(1)
            print(f"""
{r_character.info[0][1]} starts walking slowly with you""", end="")
            self.Suspension()
            print(f"\nSuddenly, {r_character.info[0][1]} takes some longer steps and disappears into the thick fog.", end="")
            self.win_chance += self.win_chance_increase
            
        else:
            
            # If the answer is wrong, the character disappears and the losing 
            # chances of the player increase
            print(r_character.info[9][1])
            print(f"""
{r_character.info[0][1]} turns and after some quick steps disappears into the thick fog.
    """, end="")
            self.lose_chance += self.lose_chance_increase            
        self.Continue(0)
        
            

    # Win method, the player has won the game and it can add a new character
    # to the labyrinth if he/she wants to
    def Win(self):

        self.Spacing(1)
        print(f"""
You start to approach him and after some steps you realize that you are out 
of the forest. Even the fog starts dissipating.

A huge man with black coat and hoodie stand before you, he's voice is powerful
and resonating.

<Uknown> 'Good job, {self.player_name} you have completed the challenge and 
you are free to go. But, before leaving you will reiceive your prize, you'll
be able to add one character to the labyrinth! You can meet this character
the next time you start the game (whenever you decide to play again)
You will choose his/her name and his/her question that will he/she will ask.
You can choose whoever you want, we have very persuasive means to convince
everyone to participate.

This is a huge privilege, many players are coming soon and you are going to
influence their exiciting life or death game.

Do you want to proceed?' 

                """)
        # Check if the player wants to add a character
        positive_answers = ["yes", "y"]
        negative_answers = ["no", "n"]
        
        # Keep asking for an answer until the player responds correctly
        accpt_answer = False
        while (accpt_answer != True):
            answer = input(prompt="> ").lower()
            if answer in positive_answers:
                
                # If the player wants to add a character the method to add one
                # is called
                self.Add_Character()
                accpt_answer = True
            elif answer in negative_answers:
                
                # If the player doesn't accept the game ends
                accpt_answer = True
                print("<Uknown> Shame on you!")
                self.Fail()
        

    # Spacing function, if the argument is 1 it adds one straight line
    def Spacing(self, line):
        if line == True or line == 1:
            print("\n___________________________________________________________________________________\n")
        else:
            print("\n\n")
        
    # Ask the player if he wants to continue, if the argument is 1 it adds one
    # straight line by calling the Spacing() method giving 1 as a an argument
    def Continue(self, line):
        if line == 1:
            self.Spacing(1)
        else:
            self.Spacing(0)
        print("<Press enter to continue>", end="")
        input()
        
    # It writes that the player is wandering inside the labyrinth
    def Wander(self):
        self.Spacing(1)        
        print("You are wandering in the thick dark forest", end="")
        self.Suspension()
        self.Approach()
        
    # The player wait for a random number of seconds, it adds some dots while
    # the player waits
    def Suspension(self):
        suspension_time = np.random.randint(self.min_wait,self.max_wait)
        for waiting_secs in range(suspension_time):
            print(".", end="")
            time.sleep(1)
    
    # Method to add a character to the characters pool
    # At the end of the method the characters pool is saved with the new 
    # character
    def Add_Character(self):
        
        # It keeps asking the player for the character characterstics until
        # he/she is satisfied with the result or he/she changes his/her mind
        # and she doesn't want to add a character anymore
        character_confirmed = False
        while character_confirmed != True:
            
            #Generate a new empty character
            new_char = Character()
            
            # Ask to the player all the charaterstics of the newly created
            # character. The only field on which there is a input control
            # is correct_answer where the input must necessarily be a number
            # between 1 and 4
            
            # Name
            print("What's the name of the new character?")
            new_char.info[0][1] = input(prompt = "> ")
            print("")
            
            # Apperance
            print(f"""
What's his/her apperance? (e.g. He's a tall man with a long white beard. 
He's very thin now but from his oversize clothes, he used to be quite fat.)""", end ="")
            new_char.info[1][1] = input(prompt = "> ")
            print("")
              
            # Question
            print("")
            print(f"""
What question will he/she ask? (e.g. How is it called the first reindeer of 
min- Santa Claus?)""")
            new_char.info[2][1] = input(prompt = "> ")
            print("")
              
            self.Spacing(1)
            print("Now you need to input 4 possible answers.")
            print("Among them there must be the correct answer\n")
              
            # Option 1
            print(f"""
What is the answer number. 1? (e.g. Rudolf)""")
            new_char.info[3][1] = input(prompt = "> ")
            print("")
            
            # Option 2
            print(f"""
What is the answer number. 2? (e.g. Lainer)""")
            new_char.info[4][1] = input(prompt = "> ")
            print("")
              
            # Option 3
            print(f"""
What is the answer number. 3? (e.g. Mandonf)""")
            new_char.info[5][1] = input(prompt = "> ")
            print("")
              
            # Option 4
            print(f"""
What is the answer number. 4? (e.g. George)""")
            new_char.info[6][1] = input(prompt = "> ")
            print("")
            
            # Correct answer
            correct_answers = ["1","2","3","4"]
            answer = 0
            while (answer not in correct_answers):
                # Correct answer
                print("What is the number of the correct answer? (1, 2, 3 or 4?)")
                answer = input(prompt = "> ")
                if answer not in correct_answers:
                    print("\nIt must be a number between 1 and 4\n")
            new_char.info[7][1] = answer
            print("")
              
            # Win message
            
            print(f"""
What will the character say if the player respond correctly? (e.g. Good job, 
son. But, this year the world is going to miss Christmas)""")
            new_char.info[8][1] = input(prompt = "> ")
            print("")
            
            # Lose message
            print("?")
            print(f"""
What will the character say if the player respond uncorrectly? (e.g. No.. I'm 
sorry, he's my friend Rudolf!)""")
            new_char.info[9][1] = input(prompt = "> ")
            print("")
              
            # Author, taken from the player_name
            new_char.info[10][1] = self.player_name
              
            positive_answers = ["yes", "y"]
            negative_answers = ["no", "n"]
            
            # Printing a summary of all the charaterstics of the newly created
            # character, givin the chance to the user to check if everything is
            # correct
            for info in new_char.info:
                print(info[0], end=":  ")
                print(info[1])
                print("\n----\n")
                
                
            # Ask if the character is corrrect 
            accpt_answer = False
            
            while (accpt_answer != True):
                print("Do you want to confirm this character?")
                answer = input(prompt= "> ").lower()
                if answer in positive_answers:
                    
                    # If the Player confirms everything is correct, submit and
                    # upload the updated characters_pool
                    self.pool.characters_pool.append(new_char)
                    self.pool.Save()
                    accpt_answer = True
                    character_confirmed = True
                elif answer in negative_answers:
                    # If the Player doesn't confirm, ask the player if she/he
                    # still want to add a character
                    print("You don't want to add a character anymore?")
                    answer = input(prompt="> ")
                    if answer in positive_answers:
                        self.Fail()
                    accpt_answer = True
                    character_confirmed = True
                else:
                    # Keep asking the question until the answer is within the 
                    # affirmative or negative answers
                    print("\nI didn't understand your answer.\n")
        
        # At the end of the method the player status is set to fainted and the
        # game ends. it's not asked if the player wants to play again
        self.fainted = True
        
    def Approach(self):
        self.Spacing(1)
        
        # Ask the user if he wants to approach someone in the fog or continue
        # looking
        print("You glimpe someone in the thick fog, do you want to approach him?")
        
        # Acceptable answer, False until the answer is confirmed to be correct
        # i.e. the player has to respond yes or no
        accpt_answer = False
        
        while (accpt_answer != True):
            approach = input(prompt="> ").lower()
            if ("yes" in approach or "approach" in approach or approach == "y") and not ("don't" in approach or "do not" in approach or "dont" in approach or "no" in approach):
                self.Rand_Encounter()
                accpt_answer = True
            elif ("no" in approach or approach == "n") or (("don't" in approach or "do not" in approach or "dont" in approach) and "approach" in approach):
                accpt_answer = True
            else:
                self.Spacing(1)
                print("You can't do that now.")
                print("Do you want to approach him?")
        if (self.fainted != True):
            self.Wander()
    
    # Fail method, it is called the player faints and the game ends
    def Fail(self):
        self.fainted = True
        self.Continue(0)
        print("You start feeling weak and you faint on the grass\n\n")
        print("The End.")
    
        
# Create an instance of the game
game = Labyrinth()

# Launch the game
game.Start()

#############################################################################




___________________________________________________________________________________


You wake up quite confused, you are laying on the grass, as
soon as you open your eyes you hear a firm voice from above you:

___________________________________________________________________________________

 <Uknown> 'What's your name?' 
> Felipe

___________________________________________________________________________________


You are outside, it's pitch dark and there's a thick fog. 
You can see some trees around you and many leaves and branches covering the sky.
It seems a really thick forest.

You see a small guy with a long black coat and a hoodie in front of you
filling out a form with your name, you imagine.

After writing he starts reading:
"Welcome Player, we will be watching you, look for someone in the forest
and answer correctly to their questions in order to escape."

As soon as he finishes to read he runs away and after a few steps he
dissolve into the fog.
        
You decide t